In [ ]:
# !pip uninstall mediapipe opencv-python==4.5.5.64
!pip uninstall mediapipe opencv-python==0.9.3.0
# !pip uninstall opencv-python
# !pip uninstall opencv-contrib-python
# !pip uninstall opencv-contrib-python-headless
# !pip uninstall -U tensorflow-estimator==2.6.0
# !pip uninstall tensorflow --upgrade

In [ ]:
!pip install mediapipe
# !pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import pandas as pd
import math
import time
from statistics import mean
import tkinter as tk
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# Create CSV file
csvfile = open('bicep_curl.csv', 'w', newline='')
csvwriter = csv.writer(csvfile)
# csvwriter.writerow(['shoulder_x', 'shoulder_y', 'elbow_x', 'elbow_y', 'wrist_x', 'wrist_y'])
csvwriter.writerow(['shoulder', 'elbow', 'wrist'])

In [279]:
# Put feedback at bottom of window
def draw_text(image, text):
    cv2.putText(image, text, (10, 480-10), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)


# Calculate angle
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 


# Calculate distance
def calculate_distance(a,b):
    x1, y1 = a
    x2, y2 = b
    
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    return distance


# Calculate velocity
def calculate_velocity(distance, time):
    velocity = distance / time
    
    return velocity


# Determine good/bad alignment
def determine_alignment(shoulder, elbow, wrist):
        
    # define threshold for straightness as 1/10 of the frame
    threshold = 0.1
    
    # calculate difference in x coodinate from shoulder to elbow and elbow to wrist
    shoulder_elbow = abs(shoulder[0] - elbow [0])
    elbow_wrist = abs(elbow[0] - wrist[0])

    # determine if alignment is good 
    if (shoulder_elbow > threshold) or (elbow_wrist > threshold):
        global alignment_error
        alignment_error = "Please keep your arm straight and inline with your shoulder."
        return 'bad'
    else:
        return 'good'

        
# Determine good/bad ROM
def determine_ROM(angle):
    # define minimum and maximum angle threshold 
    minimum = 10
    maximum = 180

    # return error/instruction
    if angle > maximum or angle < minimum:
        global rom_error
        rom_error = "You are moving beyond your safe range of motion. Please reduce the range of your movement to prevent injury."
        return 'bad'
    else:
        return 'good'
        
        
# Determine good/bad velocity
def determine_velocity(start, end, image):
    # define ideal length of rep
    minimum_duration = 1.5
    
    # calculate duration
    duration = end - start
    
    if duration < minimum_duration:
        # return error/instruction
        global velocity_error
        velocity_error = "You are moving too fast. Please slow down your movement."
        return 'bad'
    else:
        return 'good' 
    

# Calculate percentage score for alignment, ROM and velocity
def calculate_percentages(alignment_results, rom_results, velocity_results):
    
    # Calculate percentage of good alignment frames
    num_good_alignment_frames = alignment_results.count('good')
    percentage_good_alignment_frames = (num_good_alignment_frames / len(alignment_results)) * 100

    # Calculate percentage of good ROM frames
    num_good_rom_frames = rom_results.count('good')
    percentage_good_rom_frames = (num_good_rom_frames / len(rom_results)) * 100

    # Calculate percentage of good velocity reps
    num_good_velocity_reps = velocity_results.count('good')
    percentage_good_velocity_reps = (num_good_velocity_reps / len(velocity_results)) * 100
    
    return percentage_good_alignment_frames, percentage_good_rom_frames, percentage_good_velocity_reps


# Visualise percentage score as popup
def create_popup_window(percentage_good_alignment_frames, percentage_good_rom_frames, percentage_good_velocity_reps):
    # Create a popup window
    root = tk.Tk()
    root.title("Score")

    # Add labels to display the percentages
    alignment_label = tk.Label(root, text=f"Percentage of good alignment frames: {percentage_good_alignment_frames:.2f}%")
    rom_label = tk.Label(root, text=f"Percentage of good ROM frames: {percentage_good_rom_frames:.2f}%")
    velocity_label = tk.Label(root, text=f"Percentage of good velocity reps: {percentage_good_velocity_reps:.2f}%")

    # Pack the labels
    alignment_label.pack()
    rom_label.pack()
    velocity_label.pack()

    # Run the GUI loop
    root.mainloop()


In [282]:
cap = cv2.VideoCapture(0)

# Exercise variables
rep_counter = 0 
frame_counter = 0
stage = "extend"
start_time = None
end_time = None

# Define results
alignment_results = []
rom_results = []
velocity_results = []

# Initialize global error messages
global velocity_error
global rom_error
global alignment_error
velocity_error = ''
rom_error = ''
alignment_error = ''

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame_counter += 1
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                        
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160 and stage=='extend':
                start_time = time.time()
                stage = "contract"
            if angle < 30 and stage =='contract':
                end_time = time.time()
                stage="extend"
                rep_counter += 1
                
                # determine velocity of rep
                velocity = determine_velocity(start_time, end_time, image)
                velocity_results.append(velocity)
    
            # determine alignment and ROM of frame
            alignment = determine_alignment(shoulder, elbow, wrist)
            rom = determine_ROM(angle)            
    
            # calculate score and append result to list
            alignment_results.append(alignment)
            rom_results.append(rom)
            
            # Save to CSV
            csvwriter.writerow([shoulder, elbow, wrist])


        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (image.shape[1],73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(rep_counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Draw bottom box
        cv2.rectangle(image, (0, 420), (640, 480), (0, 0, 0), -1)

        # Add text to bottom box
        if velocity_error:
            velocity_text = "Note: " + velocity_error
            cv2.putText(image, velocity_text, (10, 410), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        if rom_error:
            rom_text = "Note: " + rom_error
            cv2.putText(image, rom_text, (10, 440), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        if alignment_error:
            alignment_text = "Note: " + alignment_error
            cv2.putText(image, alignment_text, (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    csvfile.close()
    cap.release()
    cv2.destroyAllWindows()
    

# Calculate percentage score for alignment, ROM and velocity
alignment_percentage, rom_percentage, velocity_percentage = calculate_percentages(alignment_results, rom_results, velocity_results)


# Create popup window
create_popup_window(percentage_good_alignment_frames, percentage_good_rom_frames, percentage_good_velocity_reps)


ZeroDivisionError: division by zero

In [281]:
# Read CSV file
df = pd.read_csv('bicep_curl.csv')
print(df)

                                     shoulder  \
0    [0.5704053640365601, 0.7455049157142639]   
1    [0.5700571537017822, 0.7455533146858215]   
2    [0.5700896978378296, 0.7466236352920532]   
3    [0.5700625777244568, 0.7470922470092773]   
4    [0.5703216195106506, 0.7481060028076172]   
..                                        ...   
128  [0.6685722470283508, 0.8541804552078247]   
129  [0.6728945374488831, 0.8582033514976501]   
130  [0.6804550886154175, 0.8613144159317017]   
131  [0.6878982782363892, 0.8714280724525452]   
132  [0.6887826919555664, 0.8671265244483948]   

                                        elbow  \
0    [0.6689730882644653, 0.8512599468231201]   
1    [0.6645397543907166, 0.8511189222335815]   
2     [0.6618248820304871, 0.851199209690094]   
3    [0.6597956418991089, 0.8506298065185547]   
4    [0.6581414937973022, 0.8510599732398987]   
..                                        ...   
128  [0.8301202654838562, 1.1395286321640015]   
129  [0.86787033081